In [1]:
# standard libraries
import math
import os
import tempfile
import json
from pathlib import Path
import pickle

# standard scientific libraries
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray, save, load
import pandas as pd
import seaborn as sns

# scikit-learn
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

# tensorflow
import tensorflow as tf
import tensorflow_addons as tfa
import keras
from keras.models import Sequential
from keras.layers import Dense,Conv1D, Conv2D, MaxPooling2D, Dropout, Flatten, Input, MaxPooling1D
from keras.optimizers import RMSprop
from keras.utils import to_categorical

2024-05-05 12:30:03.285226: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/mimer/NOBACKUP/groups/naiss2024-5-16/tloop/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/mimer/NOBACKUP/groups/naiss2024-5-16/tloop/lib/python3.10/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons sup

# Model

In [2]:
def make_model(metrics, num_classes, input_shape, output_bias=None, pool_size = 2):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    
    model = Sequential([        
        # First convolution
        Conv1D(16, 3, strides=1, activation='relu', padding='same', # TODO set as 64?
               input_shape = input_shape,
               kernel_initializer = 'he_normal',
               bias_initializer = 'zeros'),
        MaxPooling1D(pool_size = pool_size, strides = 2), # TODO set strides?
        Dropout(0.2),
        Conv1D(32, 3, strides = 1, activation = 'relu', padding = 'same',
               kernel_initializer = 'he_normal',
               bias_initializer = 'zeros'),
        MaxPooling1D(pool_size = pool_size, strides = 2),
        Dropout(0.2),
        Flatten(),
        
        # Neuron hidden layer
        Dense(int(input_shape[0]/pool_size) * 32, activation = 'relu', kernel_initializer='he_normal', bias_initializer = output_bias),
        Dropout(0.2),
        
        # Output neuron
        # Dense(1, activation='sigmoid')  # Sigmoid for binary question. It will contain a value from 0-1 where 0 for class ('not GNRA') and 1 for the other ('GNRA')
        Dense(num_classes, activation='softmax', bias_initializer=output_bias, kernel_initializer='glorot_uniform') # TODO categorical: Softmax for multiclass classification
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate = 1e-3),  # optimizer=RMSprop(lr=0.001),
        loss=keras.losses.CategoricalCrossentropy(), 
        # loss=keras.losses.BinaryCrossentropy(), # TODO remove?
        metrics=metrics)

    return model

In [3]:
def train(DATA_DIR, NUM_CLASSES, EPOCHS = 20, BATCH_SIZE = 16):
    DATA_DIR = Path(DATA_DIR)
    SET_NAME = str(DATA_DIR).split("/")[-1]
    RESULTS_FILE = Path(f"results/ANN/{SET_NAME}")
    METRICS = [ 
        keras.metrics.TruePositives(name = 'tp'), # TODO is this correct? something is wrong
        keras.metrics.FalsePositives(name = 'fp'),
        keras.metrics.TrueNegatives(name = 'tn'),
        keras.metrics.FalseNegatives(name = 'fn'),
        # keras.metrics.BinaryAccuracy(name = 'accuracy'), # TODO remove this?
        keras.metrics.CategoricalAccuracy(name='accuracy'),
        keras.metrics.Precision(name = 'precision'),
        keras.metrics.Recall(name = 'recall'),
        keras.metrics.AUC(name = 'auc', curve='roc'),
        keras.metrics.AUC(name = 'prc', curve = 'PR'), # precision-recall curve
        tfa.metrics.F1Score(name = 'f1', num_classes = NUM_CLASSES),
        tfa.metrics.MatthewsCorrelationCoefficient(name = 'mcc', num_classes = NUM_CLASSES)
    ]

    # Load dataset
    
    train_dict = np.load(DATA_DIR/"train_matrices.npz", allow_pickle=True)
    x_train = np.stack(train_dict['arr_0'], axis=0)
    y_train = np.load(DATA_DIR/"train_labels.npy", allow_pickle=True)
    y_train = to_categorical(y_train, num_classes=NUM_CLASSES)

    dev_dict = np.load(DATA_DIR/"dev_matrices.npz", allow_pickle=True)
    x_dev = np.stack(dev_dict['arr_0'], axis=0)
    y_dev = np.load(DATA_DIR/"dev_labels.npy", allow_pickle=True)
    y_dev = to_categorical(y_dev, num_classes=NUM_CLASSES)

    print("Training features shape:", x_train.shape)
    print("Training labels shape:", y_train.shape)

    print("\nDev (validation) features shape:", x_dev.shape)
    print("Dev (validation) labels shape:", y_dev.shape)

    print("\nInput shape:", x_train.shape[1:])
    print()

    INPUT_SHAPE = x_train.shape[1:]

    # Calculate class weight

    # Scaling by total/2 helps keep the loss to a similar magnitude.
    # The sum of the weights of all examples stays the same.
    y_integers = np.argmax(y_train, axis=1)
    class_weights = np.round(class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_integers), y=y_integers), 2)
    d_class_weights = dict(enumerate(class_weights))

    # Train with class weights

    model = make_model(METRICS, NUM_CLASSES, INPUT_SHAPE)

    history = model.fit(
        x_train,
        y_train,
        validation_data = (x_dev, y_dev),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        
        # The class weights go here
        class_weight=d_class_weights)
    
    # Save model
    model.save(RESULTS_FILE, overwrite=True)

    # Save history
    with open(RESULTS_FILE/"history", 'wb') as file:
        pickle.dump(history.history, file)

In [4]:
PATH = Path("../data_generation/training_data/non_homology_reduced/")
for dir in os.listdir(PATH):
    if dir[-1] != "U" and "gnravall" not in dir and os.path.isdir(PATH/dir):
        if "clusters" in dir:
            # pass
            print(dir)
            train(PATH/dir, 24)
        else:
            pass
            # train(PATH/dir, 2)

clusters_18_80_T


2024-05-05 12:30:11.823978: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38551 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:4b:00.0, compute capability: 8.0


Training features shape: (58392, 18, 5)
Training labels shape: (58392, 24)

Dev (validation) features shape: (7299, 18, 5)
Dev (validation) labels shape: (7299, 24)

Input shape: (18, 5)

Epoch 1/20
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2024-05-05 12:30:15.414994: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2024-05-05 12:30:16.323735: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


3650/3650 [==============================] - 18s 4ms/step - loss: 1.4041 - tp: 4747.0000 - fp: 10279.0000 - tn: 1332737.0000 - fn: 53645.0000 - accuracy: 0.2333 - precision: 0.3159 - recall: 0.0813 - auc: 0.8715 - prc: 0.2119 - f1: 0.0757 - mcc: 0.1007 - val_loss: 2.9657 - val_tp: 249.0000 - val_fp: 1965.0000 - val_tn: 165912.0000 - val_fn: 7050.0000 - val_accuracy: 0.0854 - val_precision: 0.1125 - val_recall: 0.0341 - val_auc: 0.8007 - val_prc: 0.1183 - val_f1: 0.1300 - val_mcc: 0.1392
Epoch 2/20
3650/3650 [==============================] - 13s 4ms/step - loss: 0.6470 - tp: 11914.0000 - fp: 17765.0000 - tn: 1325251.0000 - fn: 46478.0000 - accuracy: 0.3394 - precision: 0.4014 - recall: 0.2040 - auc: 0.9116 - prc: 0.3134 - f1: 0.1376 - mcc: 0.1683 - val_loss: 2.0871 - val_tp: 497.0000 - val_fp: 1847.0000 - val_tn: 166030.0000 - val_fn: 6802.0000 - val_accuracy: 0.1930 - val_precision: 0.2120 - val_recall: 0.0681 - val_auc: 0.9058 - val_prc: 0.2263 - val_f1: 0.1870 - val_mcc: 0.1646
Epoc

...

INFO:tensorflow:Assets written to: results/ANN/clusters_14_80_T/assets


INFO:tensorflow:Assets written to: results/ANN/clusters_14_80_T/assets


clusters_24_80_T
Training features shape: (60230, 24, 5)
Training labels shape: (60230, 24)

Dev (validation) features shape: (7529, 24, 5)
Dev (validation) labels shape: (7529, 24)

Input shape: (24, 5)

Epoch 1/20
3765/3765 [==============================] - 16s 4ms/step - loss: 1.1401 - tp: 7591.0000 - fp: 12247.0000 - tn: 1373043.0000 - fn: 52639.0000 - accuracy: 0.2735 - precision: 0.3826 - recall: 0.1260 - auc: 0.8748 - prc: 0.2506 - f1: 0.0942 - mcc: 0.1290 - val_loss: 2.1708 - val_tp: 407.0000 - val_fp: 1060.0000 - val_tn: 172107.0000 - val_fn: 7122.0000 - val_accuracy: 0.2274 - val_precision: 0.2774 - val_recall: 0.0541 - val_auc: 0.8927 - val_prc: 0.2203 - val_f1: 0.1331 - val_mcc: 0.1295
Epoch 2/20
3765/3765 [==============================] - 13s 4ms/step - loss: 0.5352 - tp: 16303.0000 - fp: 16702.0000 - tn: 1368588.0000 - fn: 43927.0000 - accuracy: 0.4175 - precision: 0.4940 - recall: 0.2707 - auc: 0.9433 - prc: 0.4138 - f1: 0.1749 - mcc: 0.2002 - val_loss: 2.0293 - val_tp

INFO:tensorflow:Assets written to: results/ANN/clusters_24_80_T/assets


INFO:tensorflow:Assets written to: results/ANN/clusters_24_80_T/assets
